In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [ ]:
# pip install torch
# !pip install torchvision


In [ ]:

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load SVHN dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_data = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)


Using downloaded and verified file: ./data\train_32x32.mat


In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False),
            nn.Tanh()  # Output is scaled between [-1, 1]
        )

    def forward(self, z):
        return self.gen(z)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # Outputs a probability of real/fake
        )

    def forward(self, x):
        return self.disc(x)


In [ ]:
z_dim = 100  # Latent vector dimension (input noise)
generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCELoss()


In [ ]:
# Imports needed for FID computation
from sklearn.metrics import precision_score, recall_score, accuracy_score
from torchvision.transforms import Normalize, Resize, Compose
from torchvision.models import inception_v3
from torchvision.models import Inception_V3_Weights
import numpy as np
from scipy.linalg import sqrtm
import torch
from torchvision.utils import save_image

# FID Calculation Function
def calculate_fid(real_images, fake_images, device):
    weights = Inception_V3_Weights.DEFAULT
    inception = inception_v3(weights=weights, transform_input=False).to(device)
    inception.eval()

    transform = Compose([
        Resize((299, 299)),  # Resize to Inception V3 size
        Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
    ])

    # Apply transformations and move to device
    real_images = torch.stack([transform(img) for img in real_images]).to(device)
    fake_images = torch.stack([transform(img) for img in fake_images]).to(device)

    # Extract features from Inception V3
    with torch.no_grad():
        real_features = inception(real_images).cpu().numpy()
        fake_features = inception(fake_images).cpu().numpy()

    # Calculate FID
    mu_real, sigma_real = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu_fake, sigma_fake = fake_features.mean(axis=0), np.cov(fake_features, rowvar=False)
    diff = mu_real - mu_fake
    covmean = sqrtm(sigma_real @ sigma_fake)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid_score = diff.dot(diff) + np.trace(sigma_real + sigma_fake - 2 * covmean)
    return fid_score

# Training Loop
num_epochs = 300
images_per_epoch = 5
batches_to_save = images_per_epoch
fid_scores = []  # Store FID scores for analysis

# Set discriminator training step multiplier
d_step = 10

for epoch in range(num_epochs):
    # Initialize variables for Precision, Recall, and Accuracy calculation
    all_labels = []
    all_preds = []

    for i, (real_images, _) in enumerate(train_loader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Train Discriminator (10 times faster)
        for _ in range(d_step):
            optimizer_d.zero_grad()

            # Real images
            real_labels = torch.ones(batch_size, 1).to(device)
            output_real = discriminator(real_images).view(-1, 1)
            loss_real = criterion(output_real, real_labels)

            # Fake images
            z = torch.randn(batch_size, z_dim, 1, 1).to(device)
            fake_images = generator(z)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            output_fake = discriminator(fake_images.detach()).view(-1, 1)
            loss_fake = criterion(output_fake, fake_labels)

            loss_d = loss_real + loss_fake
            loss_d.backward()
            optimizer_d.step()

        # Train Generator
        optimizer_g.zero_grad()
        output_fake_g = discriminator(fake_images).view(-1, 1)
        loss_g = criterion(output_fake_g, real_labels)
        loss_g.backward()
        optimizer_g.step()

        # Print losses occasionally
        if (i + 1) % 200 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], "
                  f"Discriminator Loss: {loss_d.item():.4f}, Generator Loss: {loss_g.item():.4f}")

        # Collect labels and predictions for Precision, Recall, and Accuracy
        all_labels.append(real_labels.cpu().numpy())
        all_preds.append(output_real.cpu().detach().numpy())

    # Calculate Precision, Recall, and Accuracy after each epoch
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    preds = (all_preds > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

    precision = precision_score(all_labels, preds)
    recall = recall_score(all_labels, preds)
    accuracy = accuracy_score(all_labels, preds)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}")

    # Calculate FID after each epoch
    with torch.no_grad():
        z = torch.randn(len(real_images), z_dim, 1, 1).to(device)
        fake_images = generator(z).detach().cpu()
        fid_score = calculate_fid(real_images.cpu(), fake_images, device)
        fid_scores.append(fid_score)
        print(f"Epoch [{epoch+1}/{num_epochs}] - FID Score: {fid_score:.4f}")

    # Save and print generated images every 10 epochs
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            z = torch.randn(64, z_dim, 1, 1).to(device)
            fake_images = generator(z).detach().cpu()
            save_image(fake_images, f'generated_images_epoch_{epoch+1}.png', normalize=True)
            print(f"Images saved for epoch {epoch+1}")

# Save FID scores for analysis
np.save("fid_scores.npy", fid_scores)


Epoch [1/300], Step [200/1145], Discriminator Loss: 0.0029, Generator Loss: 8.2807
Epoch [1/300], Step [400/1145], Discriminator Loss: 0.0025, Generator Loss: 8.4247
Epoch [1/300], Step [600/1145], Discriminator Loss: 0.0108, Generator Loss: 6.7537
Epoch [1/300], Step [800/1145], Discriminator Loss: 0.0079, Generator Loss: 7.0036
Epoch [1/300], Step [1000/1145], Discriminator Loss: 0.0054, Generator Loss: 7.4747
Epoch [1/300] - Precision: 1.0000, Recall: 0.9999, Accuracy: 0.9999


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\poorna raavi/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 68.5MB/s] 
c:\Users\poorna raavi\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [1/300] - FID Score: 671.6557
Epoch [2/300], Step [200/1145], Discriminator Loss: 0.0030, Generator Loss: 7.4027
Epoch [2/300], Step [400/1145], Discriminator Loss: 0.0066, Generator Loss: 7.5524
Epoch [2/300], Step [600/1145], Discriminator Loss: 0.0143, Generator Loss: 6.4431
Epoch [2/300], Step [800/1145], Discriminator Loss: 0.0004, Generator Loss: 12.4097
Epoch [2/300], Step [1000/1145], Discriminator Loss: 0.0052, Generator Loss: 7.3727
Epoch [2/300] - Precision: 1.0000, Recall: 0.9999, Accuracy: 0.9999


c:\Users\poorna raavi\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [2/300] - FID Score: 936.1640
Epoch [3/300], Step [200/1145], Discriminator Loss: 0.0009, Generator Loss: 13.9294
Epoch [3/300], Step [400/1145], Discriminator Loss: 0.0013, Generator Loss: 8.5727
Epoch [3/300], Step [600/1145], Discriminator Loss: 0.0142, Generator Loss: 6.5433
Epoch [3/300], Step [800/1145], Discriminator Loss: 0.0038, Generator Loss: 7.9625
Epoch [3/300], Step [1000/1145], Discriminator Loss: 0.0036, Generator Loss: 7.5424
Epoch [3/300] - Precision: 1.0000, Recall: 0.9999, Accuracy: 0.9999


...

Epoch [298/300] - FID Score: 404.5135
Epoch [299/300], Step [200/1145], Discriminator Loss: 0.0019, Generator Loss: 12.2144
Epoch [299/300], Step [400/1145], Discriminator Loss: 0.0064, Generator Loss: 16.5979
Epoch [299/300], Step [600/1145], Discriminator Loss: 0.0010, Generator Loss: 12.9987
Epoch [299/300], Step [800/1145], Discriminator Loss: 0.0000, Generator Loss: 15.9637
Epoch [299/300], Step [1000/1145], Discriminator Loss: 0.0008, Generator Loss: 13.2377
Epoch [299/300] - Precision: 1.0000, Recall: 1.0000, Accuracy: 1.0000


c:\Users\poorna raavi\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [299/300] - FID Score: 446.1405
Epoch [300/300], Step [200/1145], Discriminator Loss: 0.0007, Generator Loss: 15.7442
Epoch [300/300], Step [400/1145], Discriminator Loss: 0.0007, Generator Loss: 12.1528
Epoch [300/300], Step [600/1145], Discriminator Loss: 0.0043, Generator Loss: 9.8231
Epoch [300/300], Step [800/1145], Discriminator Loss: 0.0009, Generator Loss: 19.8268
Epoch [300/300], Step [1000/1145], Discriminator Loss: 0.0172, Generator Loss: 11.7601
Epoch [300/300] - Precision: 1.0000, Recall: 1.0000, Accuracy: 1.0000


c:\Users\poorna raavi\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [300/300] - FID Score: 393.5349
Images saved for epoch 300


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'